In [182]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

#student configs
data_path = "./data/"
DEBUG = True
lr = 0.5
epochs = 100
batch = 50

#given configs
NUM_INPUT = 784  # Number of input neurons
NUM_HIDDEN = 40  # Number of hidden neurons
NUM_OUTPUT = 10  # Number of output neurons
NUM_CHECK = 5  # Number of examples on which to check the gradient

# Given a vector w containing all the weights and biased vectors, extract
# and return the individual weights and biases W1, b1, W2, b2.
# This is useful for performing a gradient check with check_grad.
def unpack (w):
    W1, b1, W2, b2 = w
    return W1, b1, W2, b2

# Given individual weights and biases W1, b1, W2, b2, concatenate them and
# return a vector w containing all of them.
# This is useful for performing a gradient check with check_grad.
def pack (W1, b1, W2, b2):
    w = (W1, b1, W2, b2) # is this what he wants?
    return w

# Load the images and labels from a specified dataset (train or test).
def loadData (which):
    images = np.load("{}mnist_{}_images.npy".format(data_path,which))
    labels = np.load("{}mnist_{}_labels.npy".format(data_path,which))
    return images, labels

# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the cross-entropy (CE) loss. You might
# want to extend this function to return multiple arguments (in which case you
# will also need to modify slightly the gradient check code below).
def fCE (X, Y, w):
    (W1, b1, W2, b2) = w #unpack(w)
    z1 = compute_z(W1, X, b1)
    h1 = reLU(z1)
    z2 = compute_z(W2, h1, b2)
    Yhat = softmax(z2)
    cost = -np.mean(np.log(Yhat[Y==1]))
    return cost

def compute_z(W, x, b):
    return W.T.dot(x.T) + b

def reLU(s):
    return max(0, s)

def reLUPrime(s):
    return s>0

def predictor(X, Y, w):
    (W1, b1, W2, b2) = w #unpack(w)
    z1 = compute_z(W1, X, b1)
    h1 = reLU(z1)
    z2 = compute_z(W2, h1, b2)
    Yhat = softmax(z2)
    return Yhat

reLU = np.vectorize(reLU) # vectorizing the i/o by applying reLU func
reLUPrime = np.vectorize(reLUPrime) # vectorizing the i/o by applying reLUPrime func

def softmax(z):
    # this is how i did in DATA MINING
    # e_z = np.exp(z - np.max(z))
    # return e_z/e_z.sum()

    # this is how i did in HW3 - not sure which one is correct
    e_z = np.exp(z) 
    return e_z/np.sum(e_z, axis=0)

# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the gradient of fCE. You might
# want to extend this function to return multiple arguments (in which case you
# will also need to modify slightly the gradient check code below).
def gradCE (X, Y, w):
    (W1, b1, W2, b2) = w #unpack(w)
    z1 = compute_z(W1, X, b1) 
    h1 = reLU(z1)
    z2 = compute_z(W2, h1, b2)
    Yhat = softmax(z2)
    n = Y.shape[1]
    
    g = ((Yhat-Y).dot(W2.T)).dot(reLUPrime(z1.T))
    dW1 = g.dot(X).T
    db1 = np.mean(g, axis=0).reshape(-1,1)
    dW2 = (Yhat-Y).T.dot(h1).T
    db2 = np.mean(Yhat - Y, axis=1).reshape(-1,1)
    print(dW1.shape, db1.shape, dW2.shape, db2.shape)
    return (dW1,db1,dW2,db2)


# Given training and testing datasets and an initial set of weights/biases b,
# train the NN. Then return the sequence of w's obtained during SGD.
def train (trainX, trainY, testX, testY, w):
    (W1, b1, W2, b2) = unpack(w)
#     ws = (W1, b1, W2, b2)
    ws = w # bad ref?
    
    n = trainY.shape[0]
    print("before loop")
    for i in range(epochs):
        print(n)
        print(n//batch)
        for minibatch in range(n//batch):
            print("Training for minibatch")
            rX = trainX[minibatch*batch:(minibatch+1)*batch]
            rY = trainY[minibatch*batch:(minibatch+1)*batch]
            grads = gradCE(rX, rY, w)
            (dW1, db1, dW2, db2) = unpack(grads)
            W1 = W1 - lr*dW1
            b1 = b1 - lr*db1
            W2 = W2 - lr*dW2
            b2 = b2 - lr*db2
            w = pack(W1, b1, W2, b2)

        yhat = predictor(testX, testY, w)
        print("----------------------")
        print(np.mean(yhat.argmax(axis=0) == testY.argmax(axis=0)))
        cost = fCE(testX, testY, w)
        print(cost)
        print("----------------------")

    return w
        
            

    #     grads = gradCE(trainX[:64], trainY[:64], w)
    #     for x in range(len(ws)):
    #         ws[x] = ws[x] - lr*grads[x] # I think this should be vecotirzed outside a loop somehow
        
    #     print(ws)
        
    # return ws

In [183]:
# Load data
if "trainX" not in globals():
    trainX, trainY = loadData("train")
    testX, testY = loadData("test")

# Initialize weights randomly
W1 = 2*(np.random.random(size=(NUM_INPUT, NUM_HIDDEN))/NUM_INPUT**0.5) - 1./NUM_INPUT**0.5
b1 = 0.01 * np.ones(NUM_HIDDEN)
b1 = np.expand_dims(b1, axis=1)
W2 = 2*(np.random.random(size=(NUM_HIDDEN, NUM_OUTPUT))/NUM_HIDDEN**0.5) - 1./NUM_HIDDEN**0.5
b2 = 0.01 * np.ones(NUM_OUTPUT)
b2 = np.expand_dims(b2, axis=1)
w = pack(W1, b1, W2, b2)

if DEBUG:
    print("Shape of Image: {}".format(trainX.shape))
    print("Shape of Label: {}".format(trainY.shape))
    print("Shape of W1: {}".format(W1.shape))
    print("Shape of b1: {}".format(b1.shape))
    print("Shape of W2: {}".format(W2.shape))
    print("Shape of b2: {}".format(b2.shape))

# Check that the gradient is correct on just a few examples (randomly drawn).
idxs = np.random.permutation(trainX.shape[0])[0:NUM_CHECK]

# packed_w = gradCE(np.atleast_2d(trainX[idxs,:]), np.atleast_2d(trainY[idxs,:]), w)

# so sorry abt this mess i cant figure out with loops and this is faster for now
# grad_vector = np.append(packed_w[0].flatten(), packed_w[1].flatten())
# grad_vector = np.append(grad_vector, packed_w[2].flatten())
# grad_vector = np.append(grad_vector, packed_w[3].flatten())

# for i in packed_w:
#     grad_vector.append(i.flatten())
    
# grad_vector = np.array(grad_vector)
# print(grad_vector.shape)
# print(scipy.optimize.check_grad(fCE(np.atleast_2d(trainX[idxs,:]), np.atleast_2d(trainY[idxs,:]), w), 
#                                gradCE(np.atleast_2d(trainX[idxs,:]), np.atleast_2d(trainY[idxs,:]), w), [1]))


# print(scipy.optimize.check_grad(lambda w_: fCE(np.atleast_2d(trainX[idxs,:]), np.atleast_2d(trainY[idxs,:]), w_), \
#                                 lambda w_: gradCE(np.atleast_2d(trainX[idxs,:]), np.atleast_2d(trainY[idxs,:]), w_), \
#                                 w))

# Train the network and obtain the sequence of w's obtained using SGD.
w = (W1, b1, W2, b2)
ws = train(trainX, trainY, testX, testY, w)

Shape of Image: (55000, 784)
Shape of Label: (55000, 10)
Shape of W1: (784, 40)
Shape of b1: (40, 1)
Shape of W2: (40, 10)
Shape of b2: (10, 1)
before loop
55000
1100
Training for minibatch


ValueError: shapes (40,784) and (50,784) not aligned: 784 (dim 1) != 50 (dim 0)

In [178]:
ab = np.ones((20,))
ab = np.expand_dims(ab, axis=1)
ab.shape


(20, 1)